<a href="https://colab.research.google.com/github/sachinthadilshann/TensorFlow_for_Deep_Learning_Bootcamp_Udemy/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
 !wget https://raw.githubusercontent.com/sachinthadilshann/TensorFlow_for_Deep_Learning_Bootcamp_Udemy/15a269df7d105222999c850ce9fa956af7610972/helper_functions.py

 from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2025-09-10 04:34:57--  https://raw.githubusercontent.com/sachinthadilshann/TensorFlow_for_Deep_Learning_Bootcamp_Udemy/15a269df7d105222999c850ce9fa956af7610972/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5351 (5.2K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]   5.23K  --.-KB/s    in 0s      

2025-09-10 04:34:57 (64.0 MB/s) - ‘helper_functions.py.1’ saved [5351/5351]



In [42]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

unzip_data("nlp_getting_started.zip")

--2025-09-10 04:34:57--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.193.207, 173.194.194.207, 173.194.195.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.193.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2025-09-10 04:34:57 (131 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [43]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [44]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [45]:
train_df["target"].value_counts()

,count
target,
0,4342
1,3271


In [46]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [47]:
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [48]:
import random

random_index = random.randint(0, len(train_df)-5)

for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n ")
  print("---\n")

Target: 1 (real disaster)
Text:
Storm headed towards Idaho Falls with blowing dust &amp; winds to 60 mph.  US HWY 20 &amp; I15 look out.  #idwx http://t.co/0cR74m1Uxm
 
---

Target: 1 (real disaster)
Text:
Flooding kills 166 displace over one million in Pakistan http://t.co/iCFQl7I9oP

At least 166 people have been killed and nearly 11 lakhÛ_
 
---

Target: 0 (not real disaster)
Text:
@Flames_Nation that's the optimistic side of me. No doubt it's tough. But it can be done. I feel like the games will pivot on defense.
 
---

Target: 1 (real disaster)
Text:
@GettingLost @JennEllensBB @Muncle_jim It said they had superficial wounds and it was the pepper spray that did the most damage.
 
---

Target: 1 (real disaster)
Text:
SocialWOTS: GLOBI_inclusion: RT NRC_MiddleEast: Sandstorm engulfs caravans and tents in the #Zaatari refugee camp Û_ http://t.co/XBNLSBzzgI
 
---



In [49]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                    train_df_shuffled["target"].to_numpy(),
                                                    test_size=0.1)

In [50]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [51]:
train_sentences[:3], train_labels[:3]

(array(['Social Casualty #MTVHottest 5SOS',
        "The Blazing Elwoods @BlazingElwoods - Don't Bother Me (Doug's Song) -Tune http://t.co/QYzpB1gKmR",
        'Virgin galactic crash: early unlocking of brakes triggered structural failure - Irish Examiner http://t.co/ocMCvfDZkv'],
       dtype=object),
 array([0, 0, 1]))

## Converting text into numbers

### Text vectorization (tokenization)


In [52]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [53]:
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?


In [54]:
 #Find average number of tokens(words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [55]:
max_vocab_length = 10000 # max #words to have in our vocabulary
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [56]:
text_vectorizer.adapt(train_sentences)

In [57]:
sample_sen =  "Hello world"

vectorized_sentence = text_vectorizer([sample_sen])
vectorized_sentence

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1511,  101,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [58]:
random_sentence = random.choice(train_sentences)

print(f"Original text:\n {random_sentence}\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 @ACOUSTICMALOLEY no he was blazing it

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  1,  43,  62,  24, 502,  16,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [59]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")


Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
Bottom 5 least common words: [np.str_('parliment'), np.str_('parliamentary'), np.str_('parley\x89Ûªs'), np.str_('parksboardfacts'), np.str_('parks')]


### Creating an Embedding using an Embedding Layer

In [60]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim = 128,
                             input_length=max_length)

embedding

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding_1, built=False>

In [61]:
random_sentence = random.choice(train_sentences)

print(f"Original text:\n {random_sentence}\n\nEmbedded version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 The injuries are starting!!! Please @dallascowboys stay healthy!!! ????????????

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02364335, -0.00456934, -0.0472026 , ...,  0.02791016,
         -0.04045198,  0.00222481],
        [ 0.01979068, -0.03173499, -0.03917946, ...,  0.03393128,
          0.00215147, -0.01914532],
        [ 0.02072779,  0.0490541 , -0.01204052, ..., -0.0173587 ,
          0.0332593 ,  0.02641562],
        ...,
        [-0.01843386, -0.04774874,  0.03813802, ..., -0.02606375,
         -0.00438764,  0.03753595],
        [-0.01843386, -0.04774874,  0.03813802, ..., -0.02606375,
         -0.00438764,  0.03753595],
        [-0.01843386, -0.04774874,  0.03813802, ..., -0.02606375,
         -0.00438764,  0.03753595]]], dtype=float32)>

In [62]:
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-2.36433502e-02, -4.56934050e-03, -4.72025983e-02, -4.72336188e-02,
        -2.33277436e-02, -4.93668094e-02,  2.00965665e-02, -1.49768367e-02,
        -1.00391023e-02, -1.39389038e-02, -1.44902244e-02,  2.26221345e-02,
         8.69091600e-03, -3.71006019e-02, -2.61727702e-02,  2.71112211e-02,
         3.68307903e-03,  3.07204574e-03, -2.85702944e-03, -4.08586115e-03,
        -2.43769046e-02, -2.39376910e-02,  2.79232152e-02, -4.83252406e-02,
         3.71527188e-02, -3.10526844e-02,  4.74432223e-02,  9.88352299e-03,
         2.71405466e-02, -4.83962186e-02,  2.71735229e-02, -9.54505056e-03,
         3.94097082e-02, -1.02325454e-02,  1.10554695e-03, -9.55821201e-03,
         2.33685412e-02, -3.78108732e-02, -1.56189799e-02, -4.62749116e-02,
         4.36250083e-02, -3.98902185e-02, -3.51518169e-02, -2.24489328e-02,
         4.16805036e-02, -4.15015444e-02, -3.28963622e-02,  2.42607258e-02,
        -1.26579516e-02,  3.51776220e-0

## Modelling a text Dataset

### Model 0: Getting a baseline

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
    ("clf", MultinomialNB()) # model the text
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [64]:
baseline_score = model_0.score(val_sentences, val_labels)
baseline_score

0.8018372703412073

In [65]:
baseline_pred = model_0.predict(val_sentences)
print(baseline_pred[:20])

[0 1 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 1 0]


In [66]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [67]:
result_model_0 = calculate_results(y_true=val_labels,
                                  y_pred=baseline_pred)
result_model_0

{'accuracy': 80.18372703412074,
 'precision': 0.8196666555921015,
 'recall': 0.8018372703412073,
 'f1': 0.7949784267528707}

### Model 1: A Simple Dense Model

In [68]:
from helper_functions import create_tensorboard_callback


SAVE_DIR = "model_logs"

In [69]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string") #1dim input
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_3            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [70]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="simple_dense_model")])


Saving TensorBoard log files to: model_logs/simple_dense_model/20250910-043458
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.6377 - loss: 0.6499 - val_accuracy: 0.7769 - val_loss: 0.5288
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.8118 - loss: 0.4673 - val_accuracy: 0.7992 - val_loss: 0.4558
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.8552 - loss: 0.3665 - val_accuracy: 0.8110 - val_loss: 0.4288
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8877 - loss: 0.2882 - val_accuracy: 0.8163 - val_loss: 0.4293
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9102 - loss: 0.2417 - val_accuracy: 0.8150 - val_loss: 0.4367


In [71]:
model_1.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8011 - loss: 0.4760 


[0.4367016553878784, 0.8149606585502625]

In [72]:
model_1_pred_probs = model_1.predict(val_sentences)
#model_1_pred_probs
model_1_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[0.09811294],
       [0.9898932 ],
       [0.19786611],
       [0.07161801],
       [0.00986008],
       [0.39605677],
       [0.05437395],
       [0.9890919 ],
       [0.8084006 ],
       [0.11304175]], dtype=float32)

In [73]:
val_labels

array([0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,

In [74]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0.], dtype=float32)>

In [75]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 81.49606299212599,
 'precision': 0.8159463785434731,
 'recall': 0.8149606299212598,
 'f1': 0.8134715196885899}

In [76]:
result_model_0

{'accuracy': 80.18372703412074,
 'precision': 0.8196666555921015,
 'recall': 0.8018372703412073,
 'f1': 0.7949784267528707}

## Visualizing learned Embedding

In [77]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000,
 ['',
  '[UNK]',
  np.str_('the'),
  np.str_('a'),
  np.str_('in'),
  np.str_('to'),
  np.str_('of'),
  np.str_('and'),
  np.str_('i'),
  np.str_('is')])

In [78]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_3            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

In [80]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [81]:
# %load_ext tensorboard
# %tensorboard --logdir {SAVE_DIR}